In [1]:
%cd /Users/masha/Documents/GSOC/GSoC-Quantum-Diffusion-Model

from utils.post_training import *
from utils.statistics import *
from utils.plotting import *
from utils.angle_encoding_script import angle_encoding
from utils.haar_noising_script import apply_haar_scrambling

import numpy as np
import h5py
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import os
import math

import matplotlib.pyplot as plt
import scipy.linalg
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity as ssim_func

import pennylane as qml

/Users/masha/Documents/GSOC/GSoC-Quantum-Diffusion-Model


In [2]:
filename = "data/QG1_64x64_1k"
data_X = np.array(h5py.File(filename, "r")['X'])

print("Raw data shape:", data_X.shape)

data_X = data_X.astype(np.float32)

# log + normalize
data_X = np.log1p(data_X)
data_X = data_X / data_X.max()

# map to [-1, 1]
data_X = 2.0 * data_X - 1.0

Raw data shape: (1000, 64, 64)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev, interface="torch")
def quantum_patch_circuit(inputs):
    """
    inputs: (Batch_Size, 4) tensor
    Output: (Batch_Size, 4) tensor of expectations
    """
    # Shift input x from [-1, 1] to theta in [0, pi]
    # This ensures the encoding is monotonic and reversible
    scaled_inputs = (inputs + 1.0) * (np.pi / 2.0)
    
    # Encode 4 pixels into 2 qubits
    qml.RY(scaled_inputs[:, 0], wires=0)
    qml.RZ(scaled_inputs[:, 1], wires=0)
    qml.RY(scaled_inputs[:, 2], wires=1)
    qml.RZ(scaled_inputs[:, 3], wires=1)
    
    # Entangle
    qml.CNOT(wires=[0, 1])
    
    # Measure 4 observables to preserve information dimensionality
    # These capture both individual pixel values and their correlations
    return [qml.expval(qml.PauliZ(0)), 
            qml.expval(qml.PauliZ(1)), 
            qml.expval(qml.PauliX(0) @ qml.PauliX(1)),
            qml.expval(qml.PauliY(0) @ qml.PauliY(1))]

In [10]:
def process_images_vectorized(images):
    """
    Break 64x64 images into 2x2 patches and embed them in batches.
    images: (N, 1, 64, 64)
    returns: (N, 4, 32, 32)
    """
    N, C, H, W = images.shape
    
    patches = F.unfold(images, kernel_size=2, stride=2) 
    
    patches_flat = patches.transpose(1, 2).reshape(-1, 4)
    
    print(f"Embedding {patches_flat.shape[0]} patches on Quantum Simulator...")
    
    batch_size = 2048 
    results = []
    
    with torch.no_grad():
        for i in range(0, len(patches_flat), batch_size):
            batch = patches_flat[i : i + batch_size]
            
            # Run Circuit
            q_out = torch.stack(quantum_patch_circuit(batch), dim=1)
            results.append(q_out)
            
            if i % (batch_size * 100) == 0 and i > 0:
                print(f"Processed {i} patches...")
            
    results = torch.cat(results, dim=0) # (Total_Patches, 4)
    
    # We have 32x32 patches, each with 4 quantum features
    q_images = results.reshape(N, 32, 32, 4).permute(0, 3, 1, 2)
    
    return q_images.float()

data_tensor = torch.tensor(data_X, dtype=torch.float32).unsqueeze(1) # (N, 1, 64, 64)

quantum_data = process_images_vectorized(data_tensor)

print(f"Quantum Data Shape: {quantum_data.shape}")

Embedding 1024000 patches on Quantum Simulator...
Processed 204800 patches...
Processed 409600 patches...
Processed 614400 patches...
Processed 819200 patches...
Quantum Data Shape: torch.Size([1000, 4, 32, 32])
